In [47]:
using Pkg; Pkg.activate()

using DataFrames, CSV
using Convex, ECOS
using Distributions, Random  
using LightGraphs  
using Plots

using SparseArrays
using Base.Iterators: product

using Revise
using CarbonNetworks

# Load data

In [ ]:
function create_generation_map(nodes, resources=BASE_RESOURCES)
    n = length(nodes)
    bas = unique([node[2] for node in nodes])
    
    agg_nodes = []
    for (i, (ba, res)) in enumerate(product(bas, resources))
        push!(agg_nodes, (i, ba, res))
    end
    
    agg_map = spzeros(length(agg_nodes), length(nodes))
    for (i, ba, res) in nodes
        !(res in resources) && continue
        k = findfirst(x -> x[2:3] == (ba, res), agg_nodes)
        agg_map[k, i] = 1
    end

    return agg_nodes, agg_map
end

### Load network

In [57]:
BASE_RESOURCES = ["COL", "NG", "NUC", "OIL"]
DATAPATH = "/Users/degleris/Data/carbon_networks/"
A, gmax, pmax, f, G, nodes = parse_network_data(DATAPATH; num_generators=3)
n = length(gmax)
m = length(pmax)

@show n, m;



agg_nodes, B = create_generation_map(nodes)
B

(n, m) = (988, 1079)


304×988 SparseMatrixCSC{Float64, Int64} with 912 stored entries:
⠉⠉⠉⠑⠒⠒⠒⠒⠒⠒⠢⠤⠤⠤⠤⠤⠤⣄⣀⣀⣀⣀⣀⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠤⠤⠤⠤⠤⢄⣀⣀⣀⣀⣀⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠉⠉⠉⠉⠉⠉⠑⠒⠒⠒⠒⠒⠒⠤⠄
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠉⠉⠉⠉⠉⠉⠒⠒⠒⠒⠒⠒⠒⠤⠤⠤⠤⠤⠤⠤⣀⣀⣀⣀⣀⣀⣀
⠈⠉⠉⠉⠉⠉⠉⠒⠒⠒⠒⠒⠒⠒⠤⠤⠤⠤⠤⠤⠤⣀⣀⣀⣀⣀⣀⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠐⠒⠤⠤⠤⠤⠤⠤⢤⣀⣀⣀⣀⣀⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠉⠉⠉⠉⠉⠑⠒⠒⠒⠒⠒
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠉⠉⠉⠉⠉⠉⠉⠒⠒⠒⠒⠒⠒⠢⠤⠤⠤⠤⠤⠤⢄⣀⣀⣀

### Load cases

In [71]:
function load_case(name, agg_nodes, B; resources=BASE_RESOURCES)
    ng, n = size(B)
    
    case = DataFrame(CSV.File(CarbonNetworks.get_data_path(DATAPATH, name)))
    case = rename!(case, "Column1" => "ba")
    case = coalesce.(case, 0.0)

    bas = case[:, "ba"]
    demands = case[:, "demand"]
    for res in CarbonNetworks.SKIP_RESOURCES
        demands -= case[:, res]
    end

    # Load demands
    d = zeros(n)
    for (dem, ba) in zip(demands, bas)
        ind_d = findfirst(x -> x[2:3] == (ba, "DEMAND"), nodes)
        d[ind_d] = dem
    end  
    
    # Load generation profile
    g = zeros(ng)
    for res in resources
        for ba in bas
            ind_g = findfirst(x -> x[2:3] == (ba, res), agg_nodes)
            g[ind_g] = case[case.ba .== ba, res][1]
        end
    end
    
    return d, g, case
end

d, g, case = load_case("case_4.csv", agg_nodes, B);